# 4.3 MLE에서 파라미터 제약조건 처리법

- [1 함수](#1-함수)
- [2 데이터](#2-데이터)
- [3 순자산 분포 추정](#3-순자산-분포-추정)

최대가능도 추정(MLE: maximum likelihood estimation)에서 추정할 파라미터에 제약조건이 있는 경우가 있다. 우리의 $S_U$ 분포 모형에서는 확률밀도함수(PDF)에 들어있는 4개의 파라미터 중 $s$와 $\theta$는 항상 플러스값을 취한다. 따라서 추정에서 로그가능도 함수를 최대화하는 파라미터의 값을 수치적으로(numerically) 찾는 과정에서 $s>0$와 $\theta>0$의 제약조건을 부과하면서(bounding) 최적값을 찾아야 한다.

우리는 두 가지 방법으로 이것을 부과할 수 있다.

1. `opt.minimize` 함수의 `bounds` 옵션을 사용해서 파라미터가 취할 수 있는 범위를 직접 부과하는 방법이다. $S_U$ 분포 모형의 경우, $(m,s,\lambda,\theta)$ 4개의 파라미터가 있는데, 이 중 $s$와 $\theta$가 0보다 크다는 제약조건을 부과해야 하기 때문에 다음과 같은 식으로 `opt.minimize` 함수에 인수(argument)를 집어넣으면 된다. $m$과 $\lambda$에 대해서는 상한과 하한에 제한을 두지 않고, $s$와 $\theta$에 대해서는 하한을 $10^{-10}$으로 설정한 것이다.

        bounds=((None, None),(1e-10, None),(None, None),(1e-10, None)))

1. 항상 0보다 큰 값을 가져야 하는 $s$와 $\theta$에 대해서는 이것들 대신 먼저 $\exp(s^\prime)$와 $\exp(\theta^\prime)$을 추정한다. 즉, 먼저 $s^\prime$와 $\theta^\prime$을 추정한 다음,이것들에 지수(natural exponential)를 취하면 된다. 지수함수(natural exponential function)는 항상 0보다 큰 값을 갖는다는 사실을 이용하는 것이다.

아래 MLE를 위해 만든 함수에서 `weight_ll_SU`와 `weight_obj_SU`은 첫 번째 방법을 위한 것이고, `weight_ll_SU_e`와 `weight_obj_SU_e`은 두 번째 방법을 위한 것이다.

아래 3절의 추정 결과를 보면, 두 가지 방법의 파라미터 추정 결과가 거의 동일한 것을 확인할 수 있다.

## 1 함수

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy.optimize as opt
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

**Log Likelihood Function**

두 가지 방법을 생각해볼 수 있다. 하나는 

`s`와 `theta`가 항상 양수이기 때문에 최적화 과정에서 이를 bounding하기 위해 이들 파라미터에 `exp`를 취해서 로그가능도 함수를 만든다. 따라서 이 경우 입력 파라미터는 bounding 조건이 사라지고 모든 실수값을 취할 수 있게 된다.

In [2]:
def weight_ll_SU(x, weight, m, s, lambda_, theta):
    
    J = 1/(theta*np.sqrt((x-m)**2+s**2))
    z = (np.arcsinh((x - m)/s) - lambda_)/theta
       
    ln_pdf_vals = weight*(np.log(J) - 0.5*np.log(2*np.pi) - 0.5*(z)**2)
    log_lik_val = ln_pdf_vals.sum()
    
    return log_lik_val

In [3]:
def weight_ll_SU_e(x, weight, m, s, lambda_, theta):

    s = np.exp(s)
    theta = np.exp(theta)
    
    J = 1/(theta*np.sqrt((x-m)**2+s**2))
    z = (np.arcsinh((x - m)/s) - lambda_)/theta

    ln_pdf_vals = weight*(np.log(J) - 0.5*np.log(2*np.pi) - 0.5*(z)**2)
    log_lik_val = ln_pdf_vals.sum()
    
    return log_lik_val

**MLE Objective Function**

In [4]:
def weight_obj_SU(params, *args):

    m, s, lambda_, theta = params
    (x, weight) = args
    log_lik_val = weight_ll_SU(x, weight, m, s, lambda_, theta)
    neg_log_lik_val = -log_lik_val
    
    return neg_log_lik_val

In [5]:
def weight_obj_SU_e(params, *args):

    m, s, lambda_, theta = params
    (x, weight) = args
    
    log_lik_val = weight_ll_SU_e(x, weight, m, s, lambda_, theta)
    neg_log_lik_val = -log_lik_val
    
    return neg_log_lik_val

## 2 데이터

- [Survey of Consumer Finances (SCF)](https://www.federalreserve.gov/econres/scfindex.htm)
- [Code Book](https://sda.berkeley.edu/sdaweb/analysis/?dataset=scfcomb2022)

**압축파일 다운로드 및 압축풀기**

In [6]:
import pandas as pd
import requests
import io
import zipfile     #Three packages we'll need to unzip the data

def unzip_survey_file(year):
    import requests, io, zipfile
    import pandas as pd
    
    if int(year) <1989:
        url = 'http://www.federalreserve.gov/econresdata/scf/files/'\
        +year+'_scf'+year[2:]+'bs.zip'
    else: 
        url = 'http://www.federalreserve.gov/econres/files/scfp'+year+'s.zip'    

    url = requests.get(url)
    url_unzipped = zipfile.ZipFile(io.BytesIO(url.content))
    
    return url_unzipped.extract(url_unzipped.namelist()[0])

**2022년 데이터 로딩**

In [7]:
scf_2022 = pd.read_stata(unzip_survey_file(year = '2022'))
scf_2022.head()

,yy1,y1,wgt,hhsex,age,agecl,educ,edcl,married,kids,...,nwcat,inccat,assetcat,ninccat,ninc2cat,nwpctlecat,incpctlecat,nincpctlecat,incqrtcat,nincqrtcat
0,1,11,3027.956120,2,70,5,9,3,2,2,...,4,2,4,2,1,8,3,3,2,1
1,1,12,3054.900065,2,70,5,9,3,2,2,...,4,2,5,2,1,8,3,3,2,1
2,1,13,3163.637766,2,70,5,9,3,2,2,...,4,2,4,2,1,8,3,3,1,1
3,1,14,3166.228463,2,70,5,9,3,2,2,...,3,2,4,1,1,6,3,2,1,1
4,1,15,3235.624715,2,70,5,9,3,2,2,...,3,2,4,2,1,8,3,3,1,1


## 3 순자산 분포 추정

- 각 가구별로 5개의 관측을 **모두** 사용함
- 단위를 천달러로 변경

In [8]:
networth_2022 = scf_2022['networth']/1000
weight_2022 = scf_2022['wgt']

In [9]:
X = networth_2022
w = weight_2022/(weight_2022.sum())
X.describe().apply(lambda x: '%.2f' % x)

count      22975.00
mean       19956.40
std       110170.72
min         -555.50
25%           35.91
50%          384.50
75%         2476.11
max      2387780.90
Name: networth, dtype: object

**SU 분포 Weighted MLE**

$$\text{argmax} \sum_i w_i \log L(m,s,\lambda, \theta \mid x_i)$$

**파라미터 제약조건을 옵션(`bounds`)으로 부과하는 방식**

In [10]:
m_init = 0
s_init = 1
lambda_init = 0
theta_init = 1

params_init = np.array([m_init, s_init, lambda_init, theta_init])
res_SU = opt.minimize(
    weight_obj_SU, params_init, args=(X, w),
    method='L-BFGS-B',
    bounds=((None, None),(1e-10, None),(None, None),(1e-10, None))
)
m_MLE, s_MLE, lambda_MLE, theta_MLE = res_SU.x

print('m_MLE =',m_MLE, 's_MLE =',s_MLE, 'lambda_MLE =',lambda_MLE, 'theta_MLE =',theta_MLE) 
print("Objective function value at solution:", res_SU.fun)

m_MLE = -4.353750896476474 s_MLE = 14.875747192210131 lambda_MLE = 3.045951314930014 theta_MLE = 2.089702959886642
Objective function value at solution: 7.430483707764436


**양의 값을 갖는 파라미터에 지수(natural exponential)를 취하는 방식**

In [11]:
m_init = 0
s_init = 0
lambda_init = 0
theta_init = 0

params_init = np.array([m_init, s_init, lambda_init, theta_init])
res_SU = opt.minimize(
    weight_obj_SU_e,
    params_init, args=(X, w),
    method='L-BFGS-B'
)
m_MLE, s_MLE, lambda_MLE, theta_MLE = res_SU.x

s_MLE = np.exp(s_MLE)
theta_MLE = np.exp(theta_MLE)

print('m_MLE =',m_MLE, 's_MLE =',s_MLE, 'lambda_MLE =',lambda_MLE, 'theta_MLE =',theta_MLE)
print("Objective function value at solution:", res_SU.fun)

m_MLE = -4.353749207460307 s_MLE = 14.875830115629578 lambda_MLE = 3.045945510181015 theta_MLE = 2.089705981181354
Objective function value at solution: 7.430483707764102
